# Interfaz con Gradio para Múltiples Pipelines de Generación de Imágenes basados en Stable Diffusion

## Descripción del Proyecto

Este proyecto implementa una interfaz gráfica de usuario (GUI) interactiva utilizando la librería Gradio, diseñada para interactuar con múltiples modelos pre-entrenados de generación de imágenes basados en la arquitectura Stable Diffusion. Específicamente, la aplicación permite al usuario seleccionar entre los pipelines "Openjourney" y "Dreamlike Diffusion 1.0" para generar imágenes a partir de prompts textuales.

La interfaz facilita la configuración de parámetros clave para la generación, incluyendo el prompt principal, un prompt negativo (para guiar la generación evitando ciertos conceptos), el *Guidance Scale* (que controla cuánto influye el prompt en la imagen generada), el número de pasos de inferencia, y una semilla aleatoria (para reproducibilidad).

El pipeline de generación utiliza la librería `diffusers` de Hugging Face y optimizaciones para hardware GPU (mediante `torch_dtype=torch.float16` y `.to("cuda")`) para acelerar el proceso de inferencia. Se verifica la disponibilidad de una GPU al inicio para asegurar un rendimiento adecuado.

Este proyecto es ideal para experimentar rápidamente con diferentes estilos de generación de imágenes proporcionados por modelos de difusión, ofreciendo una herramienta accesible para la creación visual a partir de texto.

## Autor:

Martin Nicolas Serafini

## Fecha

Junio 2025

In [1]:
from IPython import get_ipython
from IPython.display import display

In [2]:
# Instalo las dependencias necesarias para los modelos de difusión, transformers, Gradio y otras utilidades.
# Utilizo --quiet para una salida más limpia durante la instalación.
!pip install --upgrade --quiet diffusers transformers accelerate gradio safetensors huggingface_hub peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 29.4 MB/s eta 0:00:00


In [3]:
import torch
import gradio as gr
from PIL import Image
from diffusers import StableDiffusionPipeline

In [4]:
# Verifico si hay una GPU disponible. Es crucial para la inferencia rápida de los modelos de difusión.
# Si no se detecta una GPU, informo al usuario que active la aceleración por hardware.
if not torch.cuda.is_available():
    raise RuntimeError("No se detectó GPU. Activa aceleración por hardware en el entorno Colab.")

In [6]:
# Carga modelos
print("Cargando modelos...")

# Cargo el modelo Openjourney, conocido por su estilo artístico tipo Midjourney.
# Especifico torch_dtype=torch.float16 para optimizar el uso de memoria y velocidad en la GPU.
# Lo muevo explícitamente al dispositivo CUDA.
openjourney_pipe = StableDiffusionPipeline.from_pretrained(
    "prompthero/openjourney",
    torch_dtype=torch.float16
).to("cuda")

# Cargo el modelo Dreamlike, que ofrece un estilo detallado y onírico.
# También utilizo float16 y lo envío a la GPU.
dreamlike_pipe = StableDiffusionPipeline.from_pretrained(
    "dreamlike-art/dreamlike-diffusion-1.0",
    torch_dtype=torch.float16
).to("cuda")

print("Modelos cargados correctamente.")

Cargando modelos...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Modelos cargados correctamente.


In [7]:
# Defino la función principal que generará imágenes utilizando los modelos cargados.
# Esta función toma los parámetros que controlan el proceso de generación.
def generar_imagen(prompt, negative_prompt, guidance, steps, seed, modelo):
    generator = torch.manual_seed(seed)

    if modelo == "openjourney":
        image = openjourney_pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            guidance_scale=guidance,
            num_inference_steps=steps,
            generator=generator
        ).images[0]

    elif modelo == "dreamlike":
        image = dreamlike_pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            guidance_scale=guidance,
            num_inference_steps=steps,
            generator=generator
        ).images[0]

    # En caso de que el modelo seleccionado no sea reconocido (situación de error),
    # genero una imagen roja simple como indicador visual del problema.
    else:
        image = Image.new("RGB", (256, 256), color="red")  # error visual

    return image

In [10]:
# Configuro la interfaz de usuario utilizando Gradio.
# Utilizo gr.Blocks para estructurar la interfaz de forma más flexible.
with gr.Blocks() as interfaz:
    # Añado un título principal y una descripción de los modelos disponibles.
    gr.Markdown("## Generador de Imágenes Multimodelo")
    gr.Markdown("Modelos: Openjourney, Dreamlike Diffusion 1.0")

    # Organizo los inputs en filas para una mejor disposición visual.
    with gr.Row():
        prompt = gr.Textbox(label="Prompt (descripción de la imagen)", placeholder="Ingresar una descripcion de la imagen deseada")
        negative_prompt = gr.Textbox(label="Prompt Negativo (opcional)", value="", placeholder="Ingresar un detalle de las cosas que se desea evitar")

    with gr.Row():
        guidance = gr.Slider(minimum=1, maximum=20, value=7.5, step=0.5, label="Guidance Scale")
        steps = gr.Slider(minimum=10, maximum=100, value=50, step=5, label="Número de pasos")

    with gr.Row():
        seed = gr.Slider(minimum=0, maximum=9999, step=1, value=42, label="Semilla aleatoria (seed)")
        modelo = gr.Dropdown(["openjourney", "dreamlike"], label="Modelo de generación", value="dreamlike")

    # Botón para iniciar el proceso de generación.
    boton = gr.Button("Generar Imagen")
    # Componente Image para mostrar el resultado.
    imagen_resultado = gr.Image(label="Resultado")

    # Configuro el comportamiento del botón: al hacer click, se llama a la función generar_imagen
    # con los inputs especificados y el resultado se muestra en imagen_resultado.
    boton.click(
        fn=generar_imagen,
        inputs=[prompt, negative_prompt, guidance, steps, seed, modelo],
        outputs=imagen_resultado
    )

# Lanzo la interfaz Gradio.
interfaz.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eee6c9b3e1ee3f9b9d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
